In [4]:
import matplotlib as mpl
mpl.__version__

'3.5.1'

In [5]:
import pandas as pd
pd.__version__

'1.4.2'

In [72]:
import numpy as np
np.__version__

'1.22.3'

In [6]:
import matplotlib.pyplot as plt

In [9]:
albums = pd.read_csv('data/albums.csv')
bands = pd.read_csv('data/bands.csv')
reviews = pd.read_csv('data/reviews.csv')

In [10]:
albums

,id,band,title,year
0,1,1,Taste the Hate,2014
1,2,3,The Bringer of Death,2014
2,3,4,Deception| Calculate What You Can't See,2005
3,4,7,Inti Huacay,1998
4,5,7,Génesis & horror,2002
...,...,...,...,...
28064,28065,37723,White Noise Paranormal,2010
28065,28066,37723,Anomalies of the Forest,2011
28066,28067,37723,Mother Earth Pantheon,2012
28067,28068,37723,Paganwinds,2014


In [145]:
rev = (reviews[['album','score']].set_index('album') *100).astype(np.int64)
rev

,score
album,
4,88
5,96
8,83
45,80
54,92
...,...
28061,89
28065,80
28065,80


In [143]:
a = []
for i in rev.index.unique():
    if len(rev.loc[i]) > 1:
        a.append(round(rev.loc[i].mean()['score']))
    else:
        a.append(round(rev.loc[i].mean()))
a[0:5]

[88, 96, 83, 80, 92]

In [147]:
data = {'album': rev.index.unique(), 'score': a}
rev = pd.DataFrame(data)
rev

,album,score
0,4,88
1,5,96
2,8,83
3,45,80
4,54,92
...,...,...
7900,28058,91
7901,28061,89
7902,28065,80
7903,28066,80


In [150]:
df = albums.merge(bands, 
                  left_on='band', 
                  right_on='id', 
                  suffixes=(None,'_r')).drop(['band', 'id_r'], 
                                             axis=1).merge(rev, 
                                                           left_on='id', 
                                                           right_on='album', 
                                                           suffixes=(None,None), 
                                                           how ='left').drop('album', 
                                                                             axis=1).set_index('id').convert_dtypes()
df

,title,year,name,country,status,formed_in,genre,theme,active,score
id,,,,,,,,,,
1,Taste the Hate,2014,('M') Inc.,United States,Unknown,2009,Death Metal,<NA>,2009-?,<NA>
2,The Bringer of Death,2014,.F.O.A.D.,France,Active,2009,Death Metal,Life and Death,2009-present,<NA>
3,Deception| Calculate What You Can't See,2005,100 Suns,United States,Active,2004,Death Metal,<NA>,2004-present,<NA>
4,Inti Huacay,1998,1917,Argentina,Active,1994,Death Metal,Dark Philosophical Poetry| Art| Religion| Psyc...,1994-present,88
5,Génesis & horror,2002,1917,Argentina,Active,1994,Death Metal,Dark Philosophical Poetry| Art| Religion| Psyc...,1994-present,96
...,...,...,...,...,...,...,...,...,...,...
28065,White Noise Paranormal,2010,Frozen Mist,United States,Active,2004,Gothic Metal (2004-2006)| Melodic Death/Gothic...,Darkness| Despair| Death| Horror| Pagan Ancestry,2004-2014| 2014-2016| 2016-present,80
28066,Anomalies of the Forest,2011,Frozen Mist,United States,Active,2004,Gothic Metal (2004-2006)| Melodic Death/Gothic...,Darkness| Despair| Death| Horror| Pagan Ancestry,2004-2014| 2014-2016| 2016-present,80
28067,Mother Earth Pantheon,2012,Frozen Mist,United States,Active,2004,Gothic Metal (2004-2006)| Melodic Death/Gothic...,Darkness| Despair| Death| Horror| Pagan Ancestry,2004-2014| 2014-2016| 2016-present,70


In [156]:
type(df.loc[1, 'theme'])

pandas._libs.missing.NAType